<a href="https://colab.research.google.com/github/kb22/Tensorflow-Course/blob/master/Classifying_colored_images_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying colored images using CNN

In this notebook, I'll create a Convolutional Neural Network with Data augmentation and Dropout to train on a collection of images of cats and dogs. The complete deep learning methodology will be followed:
1. Importing libraries
2. Loading the dataset
3. Exploratory data analysis
4. Creating the CNN model
5. Training the CNN model
6. Testing the CNN model
7. Improving the CNN model using data augmentation and dropout
8. Loss and Accuracy Analysis